# [SITCOMTN-092] - M1M3 Inertia Compensation Performance - Force Actuators Snapshot

Following [SITCOM-1115], we want to have snapshots of the forces applied to the force actuators during a slew. 

Thinking of a design, this will consist of a function that will receive:
- a topic associated with one of the forces applied to the force actuators
- a dayObs
- a slewID

Refer to the [README.md] file for details on how to set up this repository in your environment.  

[lsst-sitcom/summit_utils]: https://github.com/lsst-sitcom/summit_utils
[README.md]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md
[SITCOM-1115]: https://jira.lsstcorp.org/browse/SITCOM-1115
[SITCOMTN-092]: https://sitcomtn-092.lsst.io/

## Notebook Setup

We start setting up the notebook's variables that are propagated in our analysis.  
Here is a short description about each of them:

```
day_obs : int
    The associated day_obs of the slew event we are interested in.
slew_id : int
    The associated slew event number. Starts at 0 every night.
m1m3_topic : str
    M1M3 telemetry that we want to use for plots.
    See the notes below for more details.
summary_function : str
    A string used to represent a statistical function that we will
    apply to the telemetry of each force actuator over the time window
    associated with the TMA event. Current options are:
        mean, min, max, std
```
The available options for `m1m3_topic` are:

- [appliedAccelerationForces](https://ts-xml.lsst.io/sal_interfaces/MTM1M3.html#appliedaccelerationforces)
- [appliedAzimuthForces](https://ts-xml.lsst.io/sal_interfaces/MTM1M3.html#appliedazimuthforces)
- [appliedBalanceForces](https://ts-xml.lsst.io/sal_interfaces/MTM1M3.html#appliedbalanceforces)
- [appliedCylinderForces](https://ts-xml.lsst.io/sal_interfaces/MTM1M3.html#appliedcylinderforces)
- [appliedElevationForces](https://ts-xml.lsst.io/sal_interfaces/MTM1M3.html#appliedelevationforces)
- [appliedForces](https://ts-xml.lsst.io/sal_interfaces/MTM1M3.html#appliedforces)
- [appliedThermalForces](https://ts-xml.lsst.io/sal_interfaces/MTM1M3.html#appliedthermalforces)
- [appliedVelocityForces](https://ts-xml.lsst.io/sal_interfaces/MTM1M3.html#appliedvelocityforces)
- [forceActuatorData](https://ts-xml.lsst.io/sal_interfaces/MTM1M3.html#forceactuatordata)

In [ ]:
day_obs = 20231212
slew_id = 300
m1m3_topic = "forceActuatorData"
summary_function = "mean"

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time
from pathlib import Path

from lsst.summit.utils.efdUtils import EfdClient, getEfdData
from lsst.summit.utils.tmaUtils import getCommandsDuringEvent, TMAEvent, TMAEventMaker
from lsst.sitcom.vandv.logger import create_logger
from lsst.sitcom.vandv import m1m3
from lsst.ts.xml.tables.m1m3 import FATable

log = create_logger("SITCOMTN-092")

In [ ]:
plot_name = "SITCOMTN-092: Force Actuators Snapshot"

plot_path = Path("./plots")
plot_path.mkdir(exist_ok=True, parents=True)

event_maker = TMAEventMaker()

## Helper Functions

In [ ]:
topic = f"lsst.sal.MTM1M3.{m1m3_topic}"

if summary_function.strip().lower() == "mean":
    func = np.mean
elif summary_function.strip().lower() == "min":
    func = np.min
elif summary_function.strip().lower() == "max":
    func = np.max
elif summary_function.strip().lower() == "std":
    func = np.std

In [ ]:
# Refrieve the relevant event
evt = event_maker.getEvent(day_obs, slew_id)
if evt is None:
    raise ValueError(f"Cannot find slew {slew_id} on day-obs {day_obs}")

log.debug(
    f"Found event - day_obs={evt.dayObs} seq_num={evt.seqNum} "
    f"type={evt.type.name} end={evt.endReason.name}"
)

In [ ]:
# Query data
df = getEfdData(
    event_maker.client,
    topic,
    event=evt,
    warn=True,
)

In [ ]:
# Clean up the data
cols = [c for c in df.columns if ("xForce" in c or "yForce" in c or "zForce" in c)]
series = df[cols].apply(func)

In [ ]:
# Plot the snapshot
%matplotlib inline
fig, (ax_z, ax_y, ax_x) = plt.subplots(num=plot_name, figsize=(14, 5), ncols=3)

ax_z = m1m3.snapshot_forces_fa_map(
    ax_z, series, prefix="zForce", title=f"{m1m3_topic} - Z"
)
ax_y = m1m3.snapshot_forces_fa_map(
    ax_y, series, prefix="yForce", title=f"{m1m3_topic} - Y"
)
ax_x = m1m3.snapshot_forces_fa_map(
    ax_x, series, prefix="xForce", title=f"{m1m3_topic} - X"
)

fig.suptitle(f"{plot_name}\n day_obs={day_obs}, slew_id={slew_id}")
fig.tight_layout()
fig.savefig(plot_path / f"{plot_name}.png")
plt.show()